# Lecture 5

## Evaluator - Optimizer Workflow


In this workflow, one LLM call generates a response while another provides evaluation and feedback in a loop

#### Use-case: Real-time Database updates

In [1]:
%pip install "pymongo[srv]"

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import os
from dotenv import load_dotenv

load_dotenv()  # Load environment variables from .env file

# Corrected way to retrieve the API key
mongoDB_api_key = os.getenv("MONGODB_URI")

if not mongoDB_api_key:
    raise ValueError("❌ MONGODB_URI is not set. Check your .env file.")

print("✅ MongoDB URI Loaded Successfully")  # Debugging

uri = mongoDB_api_key

# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

✅ MongoDB URI Loaded Successfully


C:\Users\asif8\AppData\Roaming\Python\Python312\site-packages\pymongo\pyopenssl_context.py:355: CryptographyDeprecationWarning: Parsed a negative serial number, which is disallowed by RFC 5280. Loading this certificate will cause an exception in the next release of cryptography.
  _crypto.X509.from_cryptography(x509.load_der_x509_certificate(cert))


Pinged your deployment. You successfully connected to MongoDB!


In [4]:
# Corrected way to retrieve the API key
api_key = os.getenv("GOOGLE_API_KEY")

if not api_key:
    raise ValueError("GOOGLE_API_KEY not found. Make sure it's set in the .env file.")

os.environ["GOOGLE_API_KEY"] = api_key

from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest")

In [5]:
list_of_complaints = ["My phone network is weak", "My internet is slow", "My bill is too high", "I want to change my plan", "I want to cancel my internet plan", "party tonight"]

In [6]:
category_of_complaints = ["Network", "Internet", "Billing", "Internet Plan", "Cancellation"]

In [9]:
from datetime import datetime

for complain in list_of_complaints:
    print(complain)

    # First LLM call - classify the complaint into a category
    main_prompt = f"""
    You are a customer service representative. You need to classify the complaint into a categories.
    
    Complaints: {complain}
    Categories: {category_of_complaints}

    Always return the category in the list of categories.
    
    Return the output in the following format in XML format(strictly follow the format, nothing else should be there in the output):

    <category>
    Actual Category: (eg. "Network")
    </category>
    """

    classification_response = llm.invoke(main_prompt)
    print(classification_response.content)

    category = classification_response.content.strip("```xml\n").strip("</category>").strip("\n")
    print(f"Classified category: {category}")

    # Second LLM call - Evaluation classification - generate a response based on the category
    evaluation_prompt = f"""
    Context-
    The previous LLM call has classified the complaint into one the categories.

    You are a feedback generator. You need to evaluate the response of the LLM and give feedback on the response in 0 or 1.
    
    Complaint: {complain}
    Category: {category}

    List of categories to select from: {category_of_complaints}

    If no category found in the list of categories, then return 0. Otherwise return 1.
    
    Return the feedback(True or False)

    Only return True or False, no other text or explanation.
    """

    evaluation_response = llm.invoke(evaluation_prompt)
    feedback = evaluation_response.content
    print(f"Evaluation Feedback: {feedback}")

    # Store results in appropriate MongoDB collection based on evaluation
    db = client["customer_service"]  # Replace with your database name
    if feedback == "True":
        complaints_collection = db["complaints"]
    else:
        complaints_collection = db["complaints_unclassified"]

    # Create a document to store in MongoDB
    complaint_doc = {
        "complaint": complain,
        "category": category,
        "evaluation_feedback": feedback,
        "time": datetime.now()
    }

    # Insert the complain and category into the appropriate collection
    complaints_collection.insert_one(complaint_doc)


My phone network is weak
```xml
<category>
Actual Category: Network
</category>
```
Classified category: Actual Category: Network
Evaluation Feedback: True
My internet is slow
```xml
<category>
Actual Category: Internet
</category>
```
Classified category: Actual Category: Internet
Evaluation Feedback: True
My bill is too high
```xml
<category>
Actual Category: Billing
</category>
```
Classified category: Actual Category: Billing
Evaluation Feedback: True
I want to change my plan
```xml
<category>
Actual Category: Internet Plan
</category>
```
Classified category: Actual Category: Internet Plan
Evaluation Feedback: True
I want to cancel my internet plan
```xml
<category>
Actual Category: Cancellation
</category>
```
Classified category: Actual Category: Cancellation
Evaluation Feedback: True
party tonight
```xml
<category>
Actual Category: None
</category>
```
Classified category: Actual Category: None
Evaluation Feedback: False
